In [ ]:
!pip install tensorflow tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Tải dataset
(ds_train, ds_test), ds_info = tfds.load(
    'rock_paper_scissors',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [ ]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (150, 150))  # Resize ảnh về 150x150
    image = image / 255.0  # Chuẩn hóa giá trị pixel về [0, 1]
    return image, label
batch_size = 32
ds_train = ds_train.map(preprocess_image).batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess_image).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])  

In [ ]:
model = models.Sequential([
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # 3 lớp đầu ra: rock, paper, scissors
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
epochs = 10
history = model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=epochs
)

In [ ]:
test_loss, test_acc = model.evaluate(ds_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")


In [ ]:
model.save('/content/gdrive/NhanDangCuChi/rock_paper_scissors_model.h5')

In [ ]:
from google.colab import files
files.download('rock_paper_scissors_model.h5')